<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (Optimised)

**Version 2.1 - 31 Dec 2025**

**⚡ Startup time: ~6-8 minutes**

**Features:**
- Google Drive integration for persistent storage
- ComfyUI Manager already installed
- Cloudflare tunnel for external access
- Colab Secrets integration for API tokens
- Complete dependency installation
- URL appears **only when ComfyUI is fully ready**

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Add secrets (optional): Tools → Secrets → Add your API keys
3. Run all cells: Runtime → Run all
4. Wait for the cloudflared URL to appear at the bottom
5. Click the URL - ComfyUI is ready!

## 0. Configure Secrets (Optional)

Store your API keys securely using Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add your secrets:
   - `ANTHROPIC_API_KEY` - For Claude integration
   - `OPENAI_API_KEY` - For OpenAI models
   - `HF_TOKEN` - For Hugging Face private models
3. Toggle "Notebook access" for each secret

Secrets are stored in your Google account, not in the notebook file.

In [ ]:
#@title Load Secrets (if available)
import os

try:
    from google.colab import userdata
    
    secrets_found = []
    
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    
    if secrets_found:
        print(f"✓ Loaded secrets: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found. Add them in the 🔑 Secrets panel if needed.")
        
except Exception as e:
    print(f"ℹ️  Secrets not available: {e}")

## 1. Mount Google Drive

Mount your Google Drive where ComfyUI and models are already stored.

In [ ]:
#@title Mount Google Drive

print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

WORKSPACE = "/content/drive/MyDrive/ComfyUI"
%cd /content/drive/MyDrive

print(f"✓ Workspace: {WORKSPACE}")

## 2. ComfyUI Setup Check

**ComfyUI is already installed** - this just verifies it exists.

Only enable update if you need the latest ComfyUI version.

In [ ]:
#@title Check ComfyUI Installation
#@markdown Update ComfyUI to latest version? (Usually not needed)
UPDATE_COMFY_UI = False  #@param {type:"boolean"}

import os

if not os.path.exists(WORKSPACE):
    print("❌ ComfyUI not found on Drive!")
    print("Please install ComfyUI first using the standard setup notebook.")
else:
    print("✓ ComfyUI found")
    %cd {WORKSPACE}
    
    if UPDATE_COMFY_UI:
        print("  Updating ComfyUI...")
        !git pull origin master 2>/dev/null || git pull origin main
        print("  ✓ Updated!")

## 3. Install Dependencies

Installs all required Python packages from ComfyUI's requirements.txt.

In [ ]:
#@title Install All Required Dependencies

print("=== Installing Dependencies ===")

%cd {WORKSPACE}

!pip install -q -r requirements.txt
!pip install -q --upgrade comfyui-frontend-package

print("\n✓ All dependencies installed!")

## 3.5. Verify GPU

In [ ]:
import torch

print(f"PyTorch {torch.__version__}")

if torch.cuda.is_available():
    print(f"CUDA {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\n✓ GPU ready!")
else:
    print("\n❌ No GPU! Go to: Runtime > Change runtime type > T4 GPU")

## 4. ComfyUI Manager Check

In [ ]:
#@title Check ComfyUI Manager
#@markdown Update Manager to latest version? (Usually not needed)
UPDATE_MANAGER = False  #@param {type:"boolean"}

import os
manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"

if os.path.exists(manager_path):
    print("✓ ComfyUI Manager found")
    
    if UPDATE_MANAGER:
        print("  Updating Manager...")
        %cd {manager_path}
        !git pull
        print("  ✓ Updated!")
else:
    print("⚠️  Manager NOT found")

## 4.5. Install Custom Node Dependencies

In [ ]:
#@title Essential Custom Node Packages
#@markdown Quick install (5-10 seconds)
INSTALL_ESSENTIAL = True  #@param {type:"boolean"}

#@markdown ---
#@markdown **Advanced:** Full restore of ALL dependencies? (30-40 minutes!)
FULL_RESTORE = False  #@param {type:"boolean"}

if INSTALL_ESSENTIAL:
    print("=== Installing Essential Packages ===")
    
    !pip install -q ffmpeg-python deepdiff comfyui-types gguf piexif \
        transparent-background simpleeval iopath ollama anthropic segment-anything \
        GitPython civitai
    
    print("✓ Essential packages installed!")

if FULL_RESTORE:
    print("\n=== Full Restore (30-40 minutes) ===")
    %cd {WORKSPACE}
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    print("✓ Full restore complete!")

## 5. Start ComfyUI

The cloudflared URL will appear at the bottom **after all custom nodes have loaded**.

Just wait - when the URL appears, ComfyUI is ready to use!

In [ ]:
import subprocess
import threading
import time
import socket
import re
import sys

# Download cloudflared (cached between runs)
![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

# Global variable to store the URL
cloudflared_url = None
comfyui_ready = False

def start_tunnel(port=8188):
    """Start cloudflared and capture URL silently"""
    global cloudflared_url
    
    # Wait for ComfyUI server
    for _ in range(60):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break
        except:
            pass
        time.sleep(0.5)
    
    # Start cloudflared
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    # Extract URL silently
    for line in proc.stderr:
        if "trycloudflare.com" in line:
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if match:
                cloudflared_url = match.group(0)
                break

# Start tunnel in background
tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()

# Start ComfyUI and monitor output
%cd {WORKSPACE}

proc = subprocess.Popen(
    ["python", "main.py", "--dont-print-server"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Monitor ComfyUI output and print URL when ready
for line in proc.stdout:
    print(line, end='')  # Print ComfyUI output normally
    
    # Detect when custom nodes have finished loading
    if "Import times for custom nodes:" in line and not comfyui_ready:
        comfyui_ready = True
        # Wait a moment for the list to finish
        time.sleep(2)
        
        # Print the URL at the end
        if cloudflared_url:
            print("\n" + "=" * 70)
            print("\n🌐 COMFYUI IS READY! ACCESS HERE:")
            print(f"\n{cloudflared_url}\n")
            print("=" * 70 + "\n")
        else:
            print("\n⚠️  Cloudflared URL not yet available, waiting...\n")
            # Give it a few more seconds
            for _ in range(10):
                if cloudflared_url:
                    print("\n" + "=" * 70)
                    print("\n🌐 COMFYUI IS READY! ACCESS HERE:")
                    print(f"\n{cloudflared_url}\n")
                    print("=" * 70 + "\n")
                    break
                time.sleep(1)

## Tips & Troubleshooting

**Timing:**
- Dependency installation: ~2-3 min
- ComfyUI startup: ~1 min
- Custom nodes loading: ~3-5 min
- **Total: ~6-8 minutes before URL appears**

**How it works:**
- The cloudflared URL now appears **only when ComfyUI is fully loaded**
- No more 502 errors from clicking too early
- No more scrolling back to find the URL

**API Keys:**
- Store in Colab Secrets (🔑 icon)
- Enable "Notebook access"
- Auto-loaded in Section 0

**Common Issues:**
- "No GPU": Enable T4 GPU in Runtime settings
- Missing dependencies: Try FULL_RESTORE in Section 4.5

**Version Info:**
- Notebook version: 2.1 (31 Dec 2025)
- Optimised for: Google Colab with Google Drive storage
- Startup time: ~6-8 minutes (down from 40+ minutes in original)

**Storage:**
- **Permanent** (Drive): ComfyUI files, models, custom nodes, workflows
- **Temporary** (Colab VM): Python packages (reinstalled each session)